In [10]:
import extract_frames as ef
import MARS_AWS_helpers as mah
import MARS_annotation_tools as mat
import time

import sagemaker

### Extract video frames for annotation

The function `extract_frames` will sample frames from all videos found in a directory, and save those frames as jpg files.

In [ ]:
# folder containing the videos we'd like to sample for training data:
source_videos = '/path/to/videodir'

# folder where we'll save the sampled images:
image_directory = '/path/to/savedir'

First, we'll extract a set of `nframes` frames from `source_videos` and save them into `target_folder`. We're going to skip the first 100 frames of each video, when the experimenter's hand might be in the field of view.

In [ ]:
nframes = 1000
to_skip = 100

ef.extract_frames(source_videos, image_directory, n_frames, to_skip)

### Run an annotation job on AWS

You can find detailed instructions for setting up AWS to run annotation jobs [here](docs/readme_groundTruthSetup.md). For this notebook, we will assume you've set up your Lambdas, uploaded your images and the annotation template to a bucket, and created a manifest file for that bucket.

In [ ]:
# name of the s3 bucket containing the data to be annotated.
BUCKET = 'white-100'

# name of the data manifest inside that bucket.
MANIFEST = 'white_mouse_100'

# path to the template for the annotation interface.
UITEMPLATE = 's3://white-mouse-test/final_keypoint_unique_label_white.template'

Now you'll need to retrieve some Amazon Resource Names (ARNs): think of these as codes that uniquely identify different resources on AWS.

To find your `prehuman_arn`, ...

To find your `acs_arn`, ...

To find your `workteam_arn`, open the [AWS console](http://console.aws.amazon.com) and open SageMaker with the "Find Services" search bar. In the left-hand menu, click <kbd>Labeling workforces</kbd>.
* If using a public workforce, find `Mechanical_turn_workforce_team` in the team summary, and copy the provided ARN.
* If using a private workforce, click the <kbd>Private</kbd> tab at the top of the page, then scroll down to the list of private teams, and copy the ARN of the team you wish to use. Please refer to [this tutorial](docs/readme_privateWorkforce.md) to learn how to create and add users to your own private team.

In [ ]:
prehuman_arn = 'arn:aws:lambda:us-east-2:522254924758:function:serverlessrepo-aws-sagema-GtRecipePreHumanTaskFunc-xxxxxxxxxxxxx'
acs_arn = 'arn:aws:lambda:us-east-1:919226420625:function:serverlessrepo-aws-sagema-GtRecipeAnnotationConsol-xxxxxxxxxxxxx'
workteam_arn = 'arn:aws:sagemaker:us-east-2:394669845002:workteam/public-crowd/default'

Finally, we will package the task information. If you want to edit the number of annotations per image, the worker compensation, or the job description/keywords, do so here:

In [11]:
# info about the data to be annotated and the annotation interface.
task = {'BUCKET': BUCKET,         
        'MANIFEST': MANIFEST, 
        'UITEMPLATE': UITEMPLATE, }

# additional task info/instructions for annotators.
task['info'] = {'description': 'Please label each body part of interest',
                'keywords': ['pose', 'keypoints','animal'],
                'title': 'Please label each body part of interest',
                'job_name': '{}-'.format(task['BUCKET']) + str(int(time.time())), }

# Number of annotators per frame, and how much each annotator is paid per frame.
# Check AWS documentation for permissable compensation values. More workers will make your annotations
# more robust to outliers, while lower compensation often leads to noisier pose annotations.
task['price'] = {'num_workers': 5,
                 'dollars': 0,
                 'cents': 6,
                 'tenthcent': 0, }

# Insert ARNs for resources needed to run an image classification job.
task['arns'] = {'prehuman_arn': prehuman_arn,
                'acs_arn': acs_arn,
                'workteam_arn': workteam_arn, }

Now we are ready to submit the annotation job to SageMaker!

In [ ]:
# Connect to SageMaker
role = sagemaker.get_execution_role()

# Make sure the bucket is in the same region as this notebook.
mah.check_bucket_region(role, task)

# Package information/settings for the human workforce.
human_task_config = mah.configure_workforce(task)

# Format the task request.
ground_truth_request = mah.configure_workforce(task, human_task_config, role)

# Submit the task request.
sagemaker_client = boto3.client('sagemaker')
sagemaker_client.create_labeling_job(**ground_truth_request)

### Visualize manual pose annotations

### Compile pose annotations for training MARS

Now that you have collected some pose annotations, it's time to package them so you can use them to re-train MARS! To do so, we need to get your annotations into `tfrecord` format.

MARS can also handle annotations created using other systems. To make MARS tfrecords from existing DeepLabCut training data, simply set `annotation_file` to be the path to the `.csv` file containing your manual annotations.

In [ ]:
annotation_file = '/path/to/human_annotations.manifest' # or you can use a csv from DeepLabCut
keypoint_names = ['Nose','EarL','EarR','Neck','HipL','HipR','Tail']

mat.create_tfrecords(annotation_file, image_directory, keypoint_names)